In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
from utils import show_recipe
is_user_input = False

In [23]:
# 1) Load your filtered ingredients CSV
df_ing = pd.read_csv("dataset/ingredients.csv")  

# 2) Show the user a short menu of ingredients
print("Select from the following ingredients:\n")
for i, (ing, gi) in enumerate(zip(df_ing["ingredient"], df_ing["gi"])):
    print(f"{i:3d}. {ing} (GI={gi})")
print("\nEnter the indices of up to 5 ingredients, separated by commas (e.g. 0,12,47):")

# 3) Read user choice
selection = input("> ").strip()
try:
    idx_list = [int(x) for x in selection.split(",") if x.strip().isdigit()]
except ValueError:
    print("Invalid input. Please enter indices separated by commas.")
    exit(1)

idx_list = [i for i in idx_list if 0 <= i < len(df_ing)]
if not idx_list:
    print("No valid indices entered. Exiting.")
    exit(1)

chosen_ings = [df_ing.loc[i, "ingredient"] for i in idx_list]
print(f"\nYou picked: {chosen_ings}\n")

# 4) Build the <INPUT_START>…<INPUT_END> block
input_block = "<INPUT_START>" + "<NEXT_INPUT>".join(chosen_ings) + "<INPUT_END>"

# 5) Add the fixed tags around it
prompt = (
    "Give me a healthy recipe using the following ingredients (you don’t have to use all of them):"
    "<RECIPE_START>"
    f"{input_block}"
)

# print("\nFull prompt being sent to the model:")
# print(prompt)
# print("-" * 60)

is_user_input = True

Select from the following ingredients:

  0. sugar (GI=65)
  1. butter (GI=0)
  2. flour (GI=72)
  3. milk (GI=31)
  4. water (GI=0)
  5. olive oil (GI=0)
  6. brown sugar (GI=71)
  7. egg (GI=0)
  8. cream cheese (GI=0)
  9. vegetable oil (GI=0)
 10. soy sauce (GI=15)
 11. pineapple (GI=66)
 12. honey (GI=61)
 13. ground beef (GI=0)
 14. paprika (GI=0)
 15. vinegar (GI=0)
 16. nutmeg (GI=0)
 17. rice (GI=60)
 18. cheese (GI=0)
 19. ginger (GI=10)
 20. orange juice (GI=50)
 21. tomato sauce (GI=31)
 22. white wine (GI=0)
 23. coconut (GI=59)
 24. peanut butter (GI=14)
 25. carrot (GI=39)
 26. almonds (GI=0)
 27. tomato (GI=23)
 28. canola oil (GI=0)
 29. chocolate (GI=23)
 30. corn (GI=52)
 31. cumin (GI=0)
 32. sesame oil (GI=0)
 33. beans (GI=33)
 34. cucumber (GI=21)
 35. condensed milk (GI=61)
 36. green beans (GI=20)
 37. maple syrup (GI=54)
 38. oats (GI=59)
 39. hamburger (GI=66)
 40. beef (GI=0)
 41. crackers (GI=63)
 42. cranberries (GI=45)
 43. pineapple juice (GI=47)
 44. or

In [26]:
model_path = "model/finetuned_recipenlg_3"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if not is_user_input:
    prompt = (
        "Give me a healthy recipe using the following ingredients (you don't have to use all of them):"
        "<RECIPE_START>"
        "<INPUT_START>milk<NEXT_INPUT>cream<NEXT_INPUT>strawberry<NEXT_INPUT>melon<NEXT_INPUT>honey<NEXT_INPUT>salmon<INPUT_END>"
    )

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    max_length=512,
    return_attention_mask=True,
)
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=512,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True,
    eos_token_id=tokenizer.convert_tokens_to_ids("<RECIPE_END>"),
)

raw_recipe = tokenizer.decode(outputs[0])
show_recipe(raw_recipe)

Setting `pad_token_id` to `eos_token_id`:50266 for open-end generation.


## Recipe Title: Grilled Salmon With Strawberry Sauce

Inputs (base ingredients):
   - milk
   - cream
   - strawberry
   - melon
   - honey
   - salmon

Ingredients:
   - 1 1/2 cups milk
   - 1 cup half-and-half cream
   - 12 ounces strawberry, hulled and quartered
   - 8 ounces cantaloupes, peeled, seeded and cubed
   - 2 tablespoons honey
   - 4 (6-ounce) salmon fillets

Instructions:
   1. In a medium saucepan, combine the milk, half and half, strawberries and melon.
   2. Bring to a boil over medium-high heat, then reduce the heat to medium and simmer until the fruit is soft, about 10 minutes. Stir in the honey and season with salt and pepper. Set aside to cool to room temperature, stirring occasionally to prevent discolouring. Place the salmon in a resealable plastic bag and pour the cooled fruit mixture over the top. Seal the bag, pressing out as much air as possible. Refrigerate for at least 4 hours or overnight, turning once. Remove from the refrigerator 30 minutes before gril